# 전체 딥러닝 플로우 구현

In [17]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt 
%matplotlib inline

import torch
from torchvision import datasets, transforms
import tensorflow as tf

## 1. 데이터 load & 전처리

In [2]:
batch_size = 32

In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("dataset/",   # 이 위치에 있는 파일 사용
                   download=True,  # 그 위치에 파일 없으면 다운받겠다
                   train=True,
                   transform = transforms.Compose(  # 데이터를 가져올 때 transforms 안에 있는 기능들로 미리 전처리를 하겠다
                       [
                           transforms.ToTensor(),
                           transforms.Normalize(mean=(0.5, ), std=(0.5,))  # 평균, 분산이 0.5인 정규분포로 scaling 
                           
                       ]
                   )),
    batch_size=batch_size,
    shuffle=True
)

In [4]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.,), (1.))
                   ])),
    batch_size=batch_size,
    shuffle=False)

## 2.데이터 확인

PyTorch에서는 TF와 이미지를 표현하는데 있어서 차이점이 있음. 

- TF - (batch, height, width, channel)
- PyTorch - (batch, channel, height, width)

In [5]:
images, labels = next(iter(train_loader))

In [6]:
images.shape, images.dtype

(torch.Size([32, 1, 28, 28]), torch.float32)

In [7]:
torch_image = torch.squeeze(images[0])
torch_image.shape

torch.Size([28, 28])

In [8]:
image = torch_image.numpy()
label = labels[0].numpy()

```
plt.title(label)
plt.imshow(image, 'gray')
plt.show()
```
- 실행 안됨 ㅎ

## 3. 모델 정의
- 텐서플로: sequential, functionalAPI, subclass 등
- 파이토치: nn. 모듈

In [9]:
from torch import nn # 학습할 파라미터가 있는 것들
import torch.nn.functional as F # 학습할 파라미터가 없는 것들

conv = nn.Conv2d(parameta)<br>
conv(x)
- nn 안에 있는 애들은 결과가 functional $\to$ 결과를 다시 함수로 사용
<br>
<br>


F.relu(x)
F.max_pool2d(x)
- F는 궅이 functional 이 필요 없는 경우..!

In [10]:
class Net(nn.Module):
    def __init__(self):
        # forward 에서 사용되는 layer나 parameta들을 정의
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        # 실제 모델의 연산을 구현
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

```
nn.Conv2d(1, 20, 5, 1) 
1: input
20: output
5:
1:
```

```
nn.Linear(4*4*50, 500)
4*4*50: input
500: output

```

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)

In [13]:
print(model)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


## 4. 학습 로직
epoch $\to$ batch $\to$ model $\to$ loss $\to$ gradient $\to$ modelupdate

#### PyTorch에서는 model을 Training 모드로 변경 후 Training 할 수 있다. 

In [15]:
model.train()  # train 모듈

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [16]:
model.eval()  # evaluation모듈

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

#### PyTorch에도 optimizer 있음!

In [18]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=0.003)

In [21]:
for epoch in range(3):
    # Train Mode
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad() # gradient 값을 누적계산하기 때문에, 매 epoch 마다 0으로 초기화
        
        output = model(data)
        loss = F.nll_loss(output, target)  # output: 예측 // target: 정답
        
        loss.backward()
        
        optimizer.step()
        
        print("batch {} loss : {}".format(batch_idx, loss.item()), end='\r')
    
    # evaluation mode
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():  # gradient 기억 안함
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

batch 1874 loss : 0.0180473923683166534
Test set: Average Loss: 0.5050, Accuracy: 8836/10000 (88%)

batch 1874 loss : 0.0560781396925449465
Test set: Average Loss: 0.4994, Accuracy: 8755/10000 (88%)

batch 1874 loss : 0.0330062359571456976
Test set: Average Loss: 0.4763, Accuracy: 8757/10000 (88%)



```
loss = F.cross_entropy(예측, 정답)  
tf.keras.losses.CategoricalCrossentropy(정답, 예측) : 정답, 예측 모두 one-hot 인코딩 된 값
tf.keras.losses.SparseCategoricalCrossentropy(정답, 예측) : 정답으로 one-hot 인코딩 안된 label을 입력하는것도 가능
파이토치에서는 모델에 F.log_softmax() 사용, loss = F.nll_loss(예측, 정답) 을 사용하면 된다.
```